In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC # Supprt Vector Classifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,roc_curve, auc, classification_report, confusion_matrix

In [2]:
datos = pd.read_csv('clean_data.csv')

In [3]:
datos.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality_Fraud,quality_Legit,type_red,type_white
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,1,1,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0,1,1,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0,1,1,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0,1,1,0
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0,1,1,0


SE PREPARA LA DATA

In [18]:
X = datos.drop(['quality_Fraud', 'quality_Legit', 'type_red', 'type_white'], axis=1)
y = datos[['quality_Fraud', 'quality_Legit']].to_numpy()[:, 0]

In [19]:
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.25, random_state = 0)

SE NORMALIZA

In [6]:
normalizador = StandardScaler()
X_entreno = normalizador.fit_transform(X_entreno)
X_prueba = normalizador.transform(X_prueba)

In [7]:
clasificador = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
clasificador.fit(X_entreno, y_entreno)

KNeighborsClassifier()

In [8]:
y_pred = clasificador.predict(X_prueba)
y_pred_single = np.argmax(y_pred, axis=1)
y_prueba_single = np.argmax(y_prueba.values, axis=1)
print(np.concatenate((y_pred[:, 0].reshape(-1, 1), y_prueba.to_numpy()[:, 0].reshape(-1, 1)), axis=1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [9]:
mat_conf = confusion_matrix(y_prueba_single, y_pred_single)
print('Matriz de confusión: \n', mat_conf)

Matriz de confusión: 
 [[   3   55]
 [   3 1269]]


In [10]:
acc = accuracy_score(y_prueba, y_pred)
print('Accuracy: ', acc)

Accuracy:  0.956390977443609


In [11]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_entreno, y_entreno)

KNeighborsClassifier(n_neighbors=1)

In [12]:
y_pred = knn.predict(X_prueba)
classi = classification_report(y_prueba, y_pred)
print('Report KNN: \n', classi)

Report KNN: 
               precision    recall  f1-score   support

           0       0.21      0.17      0.19        58
           1       0.96      0.97      0.97      1272

   micro avg       0.94      0.94      0.94      1330
   macro avg       0.59      0.57      0.58      1330
weighted avg       0.93      0.94      0.93      1330
 samples avg       0.94      0.94      0.94      1330



REGRESIÓN LINEAL

In [13]:
modeloLog = LogisticRegression(max_iter = 500)

In [14]:
modeloLog1 = LogisticRegression(max_iter=500)
modeloLog1.fit(X_entreno, y_entreno['quality_Fraud'])

modeloLog2 = LogisticRegression(max_iter=500)
modeloLog2.fit(X_entreno, y_entreno['quality_Legit'])

LogisticRegression(max_iter=500)

In [15]:
y_entreno_combined = (y_entreno['quality_Fraud'] == 1).astype(int)

In [16]:
modeloLog = LogisticRegression(max_iter=500)
modeloLog.fit(X_entreno, y_entreno_combined)

LogisticRegression(max_iter=500)

In [17]:
y_pred = modeloLog.predict(X_prueba)

In [20]:
mat_conf = confusion_matrix(y_prueba, y_pred)
print('Matriz de confusión: \n', mat_conf)

Matriz de confusión: 
 [[1270    2]
 [  58    0]]


In [21]:
acc = accuracy_score(y_prueba, y_pred)
print('Accuracy: ', acc)

Accuracy:  0.9548872180451128


In [29]:
classi = classification_report(y_prueba, y_pred)
print('Report LINEAR REGRESSION: \n', classi)

Report LINEAR REGRESSION: 
               precision    recall  f1-score   support

           0       0.98      0.70      0.81      1272
           1       0.09      0.64      0.15        58

    accuracy                           0.69      1330
   macro avg       0.53      0.67      0.48      1330
weighted avg       0.94      0.69      0.78      1330



GRID

In [23]:
svm = SVC()
malla = SVC(C=0.001, kernel='poly', degree=4, class_weight='balanced', gamma='auto')

In [24]:
malla.fit(X_entreno, y_entreno)

SVC(C=0.001, class_weight='balanced', degree=4, gamma='auto', kernel='poly')

In [25]:
mat_conf = confusion_matrix(y_prueba, y_pred)
print('Matriz de confusión: \n', mat_conf)

Matriz de confusión: 
 [[1270    2]
 [  58    0]]


In [26]:
acc = accuracy_score(y_prueba, y_pred)
print('Accuracy: ', acc)

Accuracy:  0.9548872180451128


In [28]:
y_pred = malla.predict(X_prueba)
classi = classification_report(y_prueba, y_pred)
print('Report GRID SEARCH: \n', classi)

Report GRID SEARCH: 
               precision    recall  f1-score   support

           0       0.98      0.70      0.81      1272
           1       0.09      0.64      0.15        58

    accuracy                           0.69      1330
   macro avg       0.53      0.67      0.48      1330
weighted avg       0.94      0.69      0.78      1330

